In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [4]:
train = pd.read_csv("../input/amazon-ml-challenge-2021-hackerearth/train.csv", escapechar="\\", quoting = csv.QUOTE_NONE)


In [5]:
tdd = train.drop_duplicates()

In [6]:
t = tdd['BROWSE_NODE_ID'].value_counts()[:1000]

In [7]:
idx = t.index
idx = np.array(idx)

In [8]:
tdd1 = tdd[tdd['BROWSE_NODE_ID'].isin(idx)]

In [9]:
tdd1.shape

(2322908, 5)

In [10]:
tdd1 = tdd1.dropna()

In [11]:
tdd2 = tdd1.sample(500000, random_state = 7)

In [12]:
tdd2.shape

(500000, 5)

In [13]:
tdd1.shape

(1671610, 5)

In [14]:
test = pd.read_csv("../input/amazon-ml-challenge-2021-hackerearth/test.csv", escapechar="\\", quoting = csv.QUOTE_NONE)

In [15]:
test.shape

(110775, 5)

In [16]:
test.head()

,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,[Runs on 12 Volt DC Power or 110 Volt AC Power...,Boelter Brands
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic Landline Phones doesn't come with a ...,Panasonic
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...","[Zero Baby Girl Panties, Pack of 6, 100% Cotto...",Zero


In [17]:
import tensorflow as tf
import tensorflow_hub as hub
import logging
logging.basicConfig(level=logging.INFO)

In [18]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [19]:
import tensorflow_hub as hub
import tokenization
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [20]:
tdd2['Review'] = (tdd2['BRAND'].map(str) +' '+ tdd2['TITLE']+' '+tdd2['BULLET_POINTS']).apply(lambda row: row.strip())

In [21]:
values = {"TITLE": " ", "BRAND": " ", "BULLET_POINTS": " "}
test1 = test.fillna(value=values)

In [22]:
test1['Review'] = (test1['BRAND'].map(str) +' '+ test1['TITLE']+' '+test1['BULLET_POINTS']).apply(lambda row: row.strip())

In [23]:
idx = tdd2['BROWSE_NODE_ID'].value_counts().index

In [24]:
tdd2['BROWSE_NODE_ID']

32545      1746
1871291       5
353055     3611
322093      782
53092       289
           ... 
2814344    1538
1189133    8802
312321     2673
1890121    1024
193757        1
Name: BROWSE_NODE_ID, Length: 500000, dtype: int64

In [25]:
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
tdd3 = label_encoder.fit_transform(tdd2['BROWSE_NODE_ID'])
  
tdd3

array([500,   3, 714, ..., 636, 339,   1])

In [26]:
tdd3.shape

(500000,)

In [27]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [28]:
def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    net = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(64, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(1000, activation='softmax')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [29]:
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model, save_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D, Embedding
from tensorflow.keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Reshape, Dropout, Dense,Multiply, Dot, Concatenate,Embedding
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint

In [30]:
max_len = 50
train_input = bert_encode(tdd2.Review.values, tokenizer, max_len=max_len)
test_input = bert_encode(test1.Review.values, tokenizer, max_len=max_len)

In [31]:
train_input

(array([[  101, 12391, 12391, ...,  1012,  2017,   102],
        [  101,  2694,  2361, ...,  1060,  1011,   102],
        [  101, 20704,  8641, ...,  1060,  1044,   102],
        ...,
        [  101, 27427,  8004, ...,  1024,  1017,   102],
        [  101,  3923,  2080, ...,  1004,  2630,   102],
        [  101,  2317, 26818, ...,     0,     0,     0]]),
 array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

In [32]:
train_labels = keras.utils.to_categorical(tdd3, num_classes = 1000)

In [33]:
(train_labels==1).shape

(500000, 1000)

In [34]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 50)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 50)]         0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [35]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_history = model.fit(
    train_input, train_labels, 
    validation_split=0.2,
    epochs=6,
    callbacks=[checkpoint, earlystopping],
    batch_size=128,
    verbose="auto")

Epoch 1/6

Epoch 00001: val_accuracy improved from -inf to 0.58158, saving model to model.h5
Epoch 2/6

Epoch 00002: val_accuracy improved from 0.58158 to 0.69170, saving model to model.h5
Epoch 3/6

Epoch 00003: val_accuracy improved from 0.69170 to 0.74272, saving model to model.h5
Epoch 4/6

Epoch 00004: val_accuracy improved from 0.74272 to 0.76934, saving model to model.h5
Epoch 5/6

Epoch 00005: val_accuracy improved from 0.76934 to 0.78455, saving model to model.h5
Epoch 6/6

Epoch 00006: val_accuracy improved from 0.78455 to 0.79252, saving model to model.h5


In [ ]:
model.save("BP_5L_1000.h5")

In [ ]:
tf.keras.models.save_model(model, filepath= "BP_5l_1000_retrain.h5", overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None, save_traces=True)

In [36]:
model.load_weights('model.h5')
test_pred7 = model.predict(test_input)

In [37]:
test_pred7.shape

(110775, 1000)

In [38]:
ind = np.argwhere(test_pred7==np.amax(test_pred7,1, keepdims=True))
indices = list(map(tuple, ind))

In [39]:
len(indices)

110775

In [40]:
inde = [index[1] for index in indices]

In [41]:
len(inde)

110775

In [42]:
pred7 = label_encoder.inverse_transform(inde)

In [43]:
pred7

array([1140, 2321, 8269, ...,  800,  800,  800])

In [45]:
sub_abhyu_final = pd.DataFrame({'PRODUCT_ID': test1['PRODUCT_ID'],
                           'BROWSE_NODE_ID' : pred7})
sub_abhyu_final.to_csv("sub_abhyu_final.csv", index=False)